In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import os
from solarnet import SolarNet  # 从单独的文件导入模型

# 创建结果目录
result_dir = "D:/Project/ML/Singapore cloud project/result"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

try:
    # 1. 数据加载和预处理
    cloud_data = np.load("D:/Project/ML/Singapore cloud project/data/processed/avg_cloud_thickness.npy")
    slope_data = np.load("D:/Project/ML/Singapore cloud project/data/processed/slope_resampled.npy")

    # 确保数据形状一致
    min_shape = (min(cloud_data.shape[0], slope_data.shape[0]),
                min(cloud_data.shape[1], slope_data.shape[1]))
    cloud_data = cloud_data[:min_shape[0], :min_shape[1]]
    slope_data = slope_data[:min_shape[0], :min_shape[1]]

    # 2. 数据准备
    X = np.stack([cloud_data, slope_data], axis=0)
    X = torch.tensor(X, dtype=torch.float32).unsqueeze(0)
    Y = torch.tensor((cloud_data < 10).astype(int), dtype=torch.float32).unsqueeze(0)

    dataset = TensorDataset(X, Y)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

    # 3. 模型训练
    model = SolarNet()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print("开始训练模型...")
    epochs = 10
    for epoch in range(epochs):
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            batch_y = batch_y.unsqueeze(1)
            batch_y = nn.functional.interpolate(batch_y, size=outputs.shape[2:], mode='nearest')
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

    # 4. 保存模型
    model_path = os.path.join(result_dir, "solar_model.pth")
    torch.save(model.state_dict(), model_path)
    print(f"\n模型已保存至: {model_path}")

except Exception as e:
    print(f"\n错误：{str(e)}")
    print("\n请检查以下几点：")
    print("1. 确保数据文件存在于正确路径")
    print("2. 确保有足够的内存进行训练")
    print("3. 确保结果目录可写入")

开始训练模型...
Epoch 1/10, Loss: 0.8353
Epoch 2/10, Loss: 0.7843
Epoch 3/10, Loss: 0.7316
Epoch 4/10, Loss: 0.6525
Epoch 5/10, Loss: 0.5489
Epoch 6/10, Loss: 0.4302
Epoch 7/10, Loss: 0.3146
Epoch 8/10, Loss: 0.2180
Epoch 9/10, Loss: 0.1434
Epoch 10/10, Loss: 0.0916

模型已保存至: D:/Project/ML/Singapore cloud project/result\solar_model.pth
